In [3]:
import geopandas as gpd

gdf = gpd.read_file("twekkelerveld_buildings_energy_ageEUI_wgs84.geojson")
print(gdf.columns.tolist())


['feature_id', 'rdf_seealso', 'bag_id', 'bouwjaar', 'status', 'gebruiksdoel', 'oppervlakte_min', 'oppervlakte_max', 'aantal_verblijfsobjecten', 'vbo_\noppervlakte', 'vbo_\nstatus', 'vbo_\ngebruiksdoel', 'vbo_\nbouwjaar', 'oppervlakte_sum', 'area_share', 'el_kwh', 'gas_m3', 'gas_kwh', 'co2_kg', 'tot_kwh', 'EUI_kwh_m2', 'EUI_el_m2', 'EUI_gas_m2', 'dak_opp_m2', 'dak_opp_geschikt_m2', 'pv_panels_pot', 'pv_kwh_pot', 'pv_co2_ton_saving', 'pv_kwh_used', 'pv_coverage_el', 'el_kwh_net_pv', 'tot_kwh_net_pv', 'EUI_base_kwh_m2', 'EUI_net_pv_kwh_m2', 'co2_elec_base_kg', 'co2_saved_pv_kg', 'co2_total_base_kg', 'co2_total_after_pv_kg', 'co2_reduction_kg', 'co2_reduction_pct', 'EUI_net_pv_50', 'EUI_net_pv_100', 'co2_gas_kg', 'EUI_el_kwh_m2', 'EUI_gas_kwh_m2', 'bouwjaar_num', 'age_class', 'age_mult_raw', 'age_mult_norm', 'EUI_age_kwh_m2', 'geometry']


In [2]:
import geopandas as gpd
import pandas as pd
import re

# ---------------------------------------------------
# INPUTS
# ---------------------------------------------------
GEOJSON_IN = "twekkelerveld_buildings_energy_ageEUI_wgs84.geojson"
CSV_IN     = r"C:\Users\rakib\OneDrive - University of Twente\Desktop\MSc Thesis\Project_Idea\Idea 2\Cesiumjs_Energy\Twekklerveld_bag_pand_FIXED.csv"

# ---------------------------------------------------
# HELPERS
# ---------------------------------------------------
def normalize_bag_id(x):
    """
    Returns only the numeric pand id part, e.g.
    'NL.IMBAG.Pand.0153100000267845' -> '0153100000267845'
    '0153100000267845' -> '0153100000267845'
    '1.53E+14' -> tries to recover digits
    """
    if pd.isna(x):
        return None

    s = str(x).strip()

    # remove prefix if present
    s = s.replace("NL.IMBAG.Pand.", "").replace("NL.IMBAG.PAND.", "")

    # if scientific notation like 1.53E+14
    # try to convert to integer safely
    if re.search(r"[eE]\+?\d+", s):
        try:
            s = str(int(float(s)))
        except:
            pass

    # keep digits only (very safe)
    digits = re.sub(r"\D", "", s)
    return digits if digits != "" else None

def pick_first_existing_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

# ---------------------------------------------------
# LOAD GEOJSON (LOD1 MASTER)
# ---------------------------------------------------
gdf = gpd.read_file(GEOJSON_IN)

# Ensure WGS84
if gdf.crs is None:
    # if your file is *already* WGS84 but missing CRS metadata:
    gdf = gdf.set_crs("EPSG:4326", allow_override=True)
else:
    gdf = gdf.to_crs("EPSG:4326")

# GeoJSON join key
if "bag_id" not in gdf.columns:
    raise KeyError("GeoJSON does not contain 'bag_id'. Your columns are: " + str(list(gdf.columns)))

gdf["bag_id_clean"] = gdf["bag_id"].apply(normalize_bag_id)

# ---------------------------------------------------
# LOAD CSV (force string to avoid 1.53E+14 issues)
# ---------------------------------------------------
df = pd.read_csv(CSV_IN, dtype=str)

# Choose best ID column from your CSV screenshot:
# you have 'identificatie' (full NL.IMBAG.Pand....) AND 'pand_id_1'
csv_id_col = pick_first_existing_col(df, ["identificatie", "pand_id_1", "pand_id", "bag_id"])
if csv_id_col is None:
    raise KeyError("CSV does not contain an ID column like 'identificatie' or 'pand_id_1'. Your columns are: " + str(list(df.columns)))

df["bag_id_clean"] = df[csv_id_col].apply(normalize_bag_id)

# Make EnergyEst numeric (optional but recommended)
if "EnergyEst" in df.columns:
    df["EnergyEst"] = pd.to_numeric(df["EnergyEst"], errors="coerce")
else:
    raise KeyError("CSV missing 'EnergyEst' column. Your columns are: " + str(list(df.columns)))

# OPTIONAL: also bring solar_Ener and bouwjaar if you want
cols_to_merge = ["bag_id_clean", "EnergyEst"]
for extra in ["solar_Ener", "bouwjaar"]:
    if extra in df.columns:
        cols_to_merge.append(extra)

df_small = df[cols_to_merge].drop_duplicates(subset=["bag_id_clean"])

# ---------------------------------------------------
# JOIN
# ---------------------------------------------------
gdf_out = gdf.merge(df_small, on="bag_id_clean", how="left")

# ---------------------------------------------------
# SANITY CHECK
# ---------------------------------------------------
total = len(gdf_out)
matched = gdf_out["EnergyEst"].notna().sum()
print("Buildings in GeoJSON:", total)
print("Matched EnergyEst:", matched)
print("Match rate:", round(matched / total * 100, 2), "%")

# Show some unmatched examples (helpful)
unmatched_sample = gdf_out.loc[gdf_out["EnergyEst"].isna(), ["bag_id"]].head(10)
print("\nSample unmatched bag_id (first 10):")
print(unmatched_sample.to_string(index=False))

# ---------------------------------------------------
# EXPORT (same name, same WGS84)
# ---------------------------------------------------
OUT_FILE = "twekkelerveld_buildings_energy_ageEUI_wgs84_201225.geojson"
gdf_out = gdf_out.drop(columns=["bag_id_clean"], errors="ignore")
gdf_out.to_file(OUT_FILE, driver="GeoJSON")

print("\n✅ Saved:", OUT_FILE)


Buildings in GeoJSON: 1465
Matched EnergyEst: 1465
Match rate: 100.0 %

Sample unmatched bag_id (first 10):
Empty DataFrame
Columns: [bag_id]
Index: []

✅ Saved: twekkelerveld_buildings_energy_ageEUI_wgs84_201225.geojson


In [3]:
print("Has EnergyEst column?", "EnergyEst" in gdf_out.columns)
print("Columns (snippet):", [c for c in gdf_out.columns if "Energy" in c or c in ["bag_id","bouwjaar","solar_Ener"]])


Has EnergyEst column? True
Columns (snippet): ['bag_id', 'EnergyEst', 'solar_Ener']


In [4]:
cols_show = ["bag_id", "EnergyEst"]
for c in ["solar_Ener", "bouwjaar"]:
    if c in gdf_out.columns:
        cols_show.append(c)

matched5 = gdf_out.loc[gdf_out["EnergyEst"].notna(), cols_show].head(5)

print("\n✅ 5 matched examples (EnergyEst present):")
print(matched5.to_string(index=False))



✅ 5 matched examples (EnergyEst present):
          bag_id  EnergyEst solar_Ener
0153100000267845    277.764    277.764
0153100000216308    230.848    230.848
0153100000244124    238.554    238.554
0153100000226792    228.401    228.401
0153100000254657    228.856    228.856
